#查看GPU

In [1]:
!nvidia-smi

Sun Apr 14 10:31:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#安裝需要的BERT套件

In [1]:
!pip install transformers==4.29.0 -U
!pip install nlp -U
!pip install torch==2.0.0 -U
!pip install transformers accelerate

  Using cached nlp-0.4.0-py3-none-any.whl (1.7 MB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Using cached xxhash-3.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Using cached torch-2.0.0-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.3 MB/s eta 0

#載入套件

In [2]:
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
# BertForSequenceClassification 用於序列分類任務的BERT模型
# BertTokenizerFast 用于快速分詞的 BERT 分詞器
# Trainer 用於訓練模型
# TrainingArguments 用於指定訓練參數

from nlp import load_dataset, Dataset
# load_dataset 用於從各種來源加載數據集
# Dataset 用於表示一個數據集

import nlp
import torch
# torch 用於深度學習和張量操作

import random
# random 用於隨機操作，如選擇隨機樣本或設置隨機種子

import pandas as pd
# pandas 用於數據操作和處理

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# sklearn.metrics 用於計算模型的準確度
# precision_recall_fscore_support 用於計算模型的精度、召回率和 F1 分數

#抓取訓練資料

In [3]:
!wget https://github.com/shhuangmust/AI/raw/112-1/train.csv # 下載文件

--2024-04-14 10:36:33--  https://github.com/shhuangmust/AI/raw/112-1/train.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv [following]
--2024-04-14 10:36:33--  https://raw.githubusercontent.com/shhuangmust/AI/112-1/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388895 (380K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 379.78K  --.-KB/s    in 0.01s   

2024-04-14 10:36:33 (29.0 MB/s) - ‘train.csv’ saved [388895/388895]



#讀取訓練資料集


In [4]:
df = pd.read_csv('train.csv')
# 使用pandas庫讀取名为'train.csv'的CSV文件，並将數據存在pandas DataFrame中

dataset = Dataset.from_pandas(df)
# 使用NLP庫中的 Dataset 模塊，將pandas DataFrame轉换为NLP庫的Dataset對象

In [5]:
df.sample(10) # 隨機抽取10個樣本

,ID,Name,Description,label
29,29,凍齡美肌懶人包(30天份),「凍齡美肌懶人包(30天份)、熱賣預購中-雪櫻美白懶人包、脂肪再見懶人包(30天份)、輪班星...,1
307,307,緣授茶,「調節生育機能的茶…一直有的經痛減輕了都不用吃止痛藥，有時不規律的出血或血塊都改善了。…已經...,1
379,379,長庚研發龜鹿四珍帝王養生專案,「保留活性100%...人體吸收100%...可以宣稱療效的食品…美國藥物管理局頒發認證.....,1
610,610,【GNC健安喜】紅茄膠囊60顆(蕃茄紅素),每份(2顆)含有30mg的專利Lyc-O-Mato?蕃茄紅素。自天然的成熟紅蕃茄萃取而得。幫...,0
576,576,一家人塑纖菌,「…姐姐是纖女…三餐不定時，三餐老是在外，爆肥體態等著你!?肥胖身型健康藏危機，三高風險跟著...,1
26,26,GNC健安喜LACFull-Cal優鎂鈣,「鈣太少可能會造成的症狀包括手指麻木、刺痛、肌肉痙攣、抽搐、嗜睡、食慾降低，心律不整。如果不...,1
84,84,青梅精組合,「調節腸胃道功能…達到體內酸鹼平衡…讓血液變成鹼性…史上超強的鹼性食品，具有殺菌、排毒、淨血...,1
418,418,減脂餅乾套組,「數據會說話!每天讓你看減脂真相!減脂前後對照2天減脂5.3KG減重4.8KG7天減11.1...,1
683,683,天然纖體錠(安麗)CarbBloc,瓦解美食澱醣嗜吃甜食、愛吃米麵的人有福了！天然纖體錠為美食澱糖把關，讓你享受美食無負擔。,0
347,347,舜鶴鮮果複合酵素,「高纖酵素的營養成份:...吸收腸道中大量有毒物質，能協助有益菌繁殖，抑制有害菌...除可防...,1


#載入原始模型

In [6]:
model = BertForSequenceClassification.from_pretrained('hfl/chinese-bert-wwm-ext')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

#載入分詞模型

In [7]:
tokenizer = BertTokenizerFast.from_pretrained('hfl/chinese-bert-wwm-ext')

tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

#設定執行參數

In [8]:
RANDOM_SEED = 5  # 設置隨機種子
MAX_LEN = 512   # 設置最大序列長度，這是BERT模型在處理文本時能夠接受的最大輸入長度
EPOCHS=5      # 設置訓練的週期數
BATCH_SIZE = 8  # 設置訓練時的樣本數量

#把資料集分成訓練、測試、驗證

In [15]:
shuffled_ds = dataset.shuffle(RANDOM_SEED)
# shuffled_ds 對數據集進行洗牌，並使用給定的隨機種子

split_ds = shuffled_ds.train_test_split(test_size=0.2)
# split_ds 將洗牌後的數據集分割為訓練集和測試集，測試集大小為數據集的20%

train_dataset = split_ds['train']
# train_dataset 從分割後的數據集中提取訓練集

test_val_dataset = split_ds['test']
# test_val_dataset從分割後的數據集中提取測試集

split_tv = test_val_dataset.train_test_split(test_size=0.5)
# split_tv 將測試集進一步分割為測試集和驗證集，驗證集大小為測試集的一半

test_dataset = split_tv['train']
# test_dataset 從分割後的數據集中提取最終的測試集

val_dataset = split_tv['test']
# val_dataset 從分割後的數據集中提取驗證集

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

#取一筆出來瞧瞧(已經打亂)

In [16]:
train_dataset[2]  # 輸出訓練集中第三樣本

{'Description': '生活難免有時感到失衡與挫敗，特別是心情低落突然來襲時。有時候您只需要將自己暫時從忙碌的生活壓力與繁瑣的思緒中解脫出來，享受片刻安寧。愛尚它益善舒寧配方粉末食品為您的生活找回些許平靜。愛尚它益善舒寧配方粉末食品含有特殊配製的多種維生素、胺基酸與礦物質等成分，並採用便利的單包包裝。此配方含多種關鍵成分，讓您沉澱心情，幫助入睡，精神氣色自然好!',
 'ID': 665,
 'Name': '愛尚它益善?舒寧配方粉末食品',
 'label': 0}

In [11]:
tokenizer.tokenize("體重跟體脂都有在持續緩慢的下降中")# 使用事先加載的 BERT 分詞器將句子進行分詞

['體',
 '重',
 '跟',
 '體',
 '脂',
 '都',
 '有',
 '在',
 '持',
 '續',
 '緩',
 '慢',
 '的',
 '下',
 '降',
 '中']

#定義分詞器

In [17]:
def tokenize(batch):
  return tokenizer(batch['Description'], max_length=MAX_LEN, padding=True, truncation=True)
# 將批次中的 'Description' 字段進行分詞，設置最大長度、填充和截斷

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
# map為訓練、測試和驗證數據集進行分詞處理
# batched=True 為批量處理
# batch_size 設置為數據集的長度，一次對整個數據集進行分詞

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
# 設置數據集的格式為 PyTorch 張量，並選擇需要的列

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

#定義評測準則，訓練器


In [18]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # 從預測結果中提取標籤和預測值

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    # 計算精確率、召回率和 F1 分數

    acc = accuracy_score(labels, preds)
    # 計算準確率

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
    # 返回度量結果的字典，包括準確率、F1 分數、精確率和召回率

training_args = TrainingArguments(
    output_dir='./results',     # 設置訓練參數
    num_train_epochs=EPOCHS,    # 訓練的周期數
    per_device_train_batch_size=BATCH_SIZE,   # 每個設備上的訓練批量大小
    per_device_eval_batch_size=BATCH_SIZE,   # 每個設備上的評估批量大小
    warmup_steps=50,        # 熱身步驟數
    weight_decay=0.01,       # 權重衰減
    # evaluate_during_training=True,
    logging_dir='./logs',     # 記錄的輸出目錄
)
# 創建 Trainer 對象，進行模型訓練和評估
trainer = Trainer(
    model=model,              # 訓練的模型
    args=training_args,          # 訓練參數
    compute_metrics=compute_metrics,   # 計算度量的方法
    train_dataset=train_dataset,      # 訓練數據集
    eval_dataset=test_dataset       # 評估數據集
)

#開始訓練

In [19]:
trainer.train()    # 執行訓練過程
trainer.evaluate()   # 執行模型評估

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


{'eval_loss': 0.13137751817703247,
 'eval_accuracy': 0.9787234042553191,
 'eval_f1': 0.9833333333333333,
 'eval_precision': 1.0,
 'eval_recall': 0.9672131147540983,
 'eval_runtime': 3.0873,
 'eval_samples_per_second': 30.447,
 'eval_steps_per_second': 3.887,
 'epoch': 5.0}

#找一個範例來測試

In [20]:
print(df.iloc[220]['label'], df.iloc[220]['Description'])   # 輸出索引 220 的數據集中的標籤和描述
print(df.iloc[578]['label'], df.iloc[578]['Description'])   # 輸出索引 578 的數據集中的標籤和描述

1 「增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常」
0 ·???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品。


#設定測試文字

In [21]:
#test_text = '增強免疫力...護眼明目...抗氧化...保肝...皮膚：...保護皮膚，減緩皮屑問題，降低敏感...毛髮……促進毛髮亮麗...消化系統」「強化免疫系統，有助調整過敏體質...清除自由基延緩老化...調整腸胃道機能...調理皮膚問題...諾麗果含有許多強大的抗氧化劑」「阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外」「它能維護身體細胞組織正常運作，阻止或減少病痛的發生，並迅速修補已受傷的細胞，將代謝殘渣或毒素排出體外，使身體恢復正常'
test_text = '???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品'
# 定義測試文本

MAX_LEN = 512  # 最大序列長度

# 使用 tokenizer.encode_plus 對測試文本進行編碼
encoded_review = tokenizer.encode_plus(
    test_text,
    max_length=MAX_LEN,        # 設置最大序列長度
    add_special_tokens=True,      # 添加特殊標記（例如CLS和SEP標記）
    return_token_type_ids=False,    # 不返回 token 類型 ID
    return_attention_mask=True,    # 返回注意力掩碼
    return_tensors='pt',        # 以 PyTorch 張量格式返回
    truncation=True           # 對超過 max_length 的文本進行截斷
)

#進行測試

In [22]:
test_model = model.to('cuda')
# 將模型移到 CUDA（GPU）設備，以便進行加速計算

input_ids = encoded_review['input_ids'].to('cuda')
attention_mask = encoded_review['attention_mask'].to('cuda')
# 將編碼後的輸入 ID 和注意力掩碼移動到 CUDA（GPU）設備

output = test_model(input_ids, attention_mask)
# 使用測試模型進行推論

result = torch.argmax(output[0][0])
# 獲取模型輸出結果中的最大值索引

classnames = ['合格', '不合格']
# 定義類別名稱列表

print(f'廣告詞: {test_text}')
print(f'檢測結果: {classnames[result]}')
# 輸出廣告詞和檢測結果

廣告詞: ???????每份含有30mg輔Q10。輔Q10可參與維生素E再生，兩者具協同作用，可促進新陳代謝。保護循環順暢維持健康。暢銷日本的最佳美容聖品
檢測結果: 合格
